# Get a list of all databases with the *.db extension

In [ ]:
import os
import os.path
title = "database.list"
f= open(title,"w");f.close()
count=0
for dirpath, dirnames, filenames in os.walk("/home"):
    filenames = [f for f in filenames if not f[0] == '.']
    dirnames[:] = [d for d in dirnames if not d[0] == '.']
    for filename in [f for f in filenames if f.endswith(".db")]:
        count=count+1
        Path = os.path.join(dirpath, filename)
        with open(title, 'a') as outfile:
            path = Path+"\n"
            outfile.write(path)

In [ ]:
Here is an example of the database.list contents:

/home/jack/unique-End2.db
/home/jack/rollerball/Library/AssetVersioning.db
/home/jack/rollerball/Library/ShaderCache.db
/home/jack/nltk_data/corpora/city_database/city.db
/home/jack/nltk_data/corpora/toolbox/iu_mien_samp.db
/home/jack/nltk_data/corpora/toolbox/MDF/MDFSampl.db
....... 

# Identify the Tables and Columns in Unknown DatabAses

In [ ]:
I had over 200 different files. May I had no idea what was in them. To open them and look at the contents
I had to know the tables and columns. This next cell opens a database identifies the table and columns

In [57]:
import sqlite3
# This reads an unknown Sqlite3 Database, thens creates or reuses a database
# to store the information retrieved
def Sdbinfo(unknown, storage):
    con = sqlite3.connect(storage)
    co = con.cursor()
    co.execute("""
    CREATE VIRTUAL TABLE IF NOT EXISTS data 
    USING FTS4(unknown, row, columns);
    """)
    con.commit()
    
    conn = sqlite3.connect(unknown)
    conn.text_factory = str
    c = conn.cursor()
    res = c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    row = c.fetchone()
    row = str(row)
    row = row.replace("(","");row = row.replace(",)","")
    row = row.replace("'","");row = row.replace(",","")
    cur = c.execute("select * from \"%s\"" %  row)
    columns = [description[0] for description in cur.description]
    col = str(columns)
    col = col.replace("[","");col = col.replace("]","")
    coll = col.replace("'","")
    print row, coll
    co.execute("INSERT into data (unknown, row, columns) values (?,?,?)",(unknown, row, coll))
    con.commit()
    conn.close()
    conn.close()
    return columns

# picked out a single file to test
unknown = "/home/jack/nltk_data/corpora/city_database/city.db"
storage = "Store.db"
Sdbinfo(unknown, storage)

city_table City, Country, Population


['City', 'Country', 'Population']

# Now to test it the information entered into the database.

In [63]:
import sqlite3
conn = sqlite3.connect("Store.db")
c = conn.cursor()
for row in c.execute("SELECT ROWID, * FROM data"):
    print "Row id:              ", row[0]
    print "File path and Name:  ", row[1]
    print "Print Table:         ", row[2]
    print "Columns:             ", row[3]

Row id:               1
File path and Name:   /home/jack/nltk_data/corpora/city_database/city.db
Print Table:          city_table
Columns:              City, Country, Population


# Verify the table and columns are correct

In [6]:
import sqlite3
conn = sqlite3.connect("/home/jack/nltk_data/corpora/city_database/city.db")
c = conn.cursor()
count = 0
for row in c.execute("SELECT ROWID, * FROM city_table"):
    print "Row id:        ", row[0]
    print "City:          ", row[1]
    print "County:        ", row[2]
    print "Population:    ", row[3]
    print "---------------------"
    count = count +1
    if count >4:
        break

Row id:         1
City:           athens
County:         greece
Population:     1368
---------------------
Row id:         2
City:           bangkok
County:         thailand
Population:     1178
---------------------
Row id:         3
City:           barcelona
County:         spain
Population:     1280
---------------------
Row id:         4
City:           berlin
County:         east_germany
Population:     3481
---------------------
Row id:         5
City:           birmingham
County:         united_kingdom
Population:     1112
---------------------


## Print the whole database

In [64]:
import sqlite3
conn = sqlite3.connect("/home/jack/nltk_data/corpora/city_database/city.db")
c = conn.cursor()
for row in c.execute("SELECT ROWID, * FROM city_table"):
    print "Row id:        ", row[0]
    print "City:          ", row[1]
    print "County:        ", row[2]
    print "Population:    ", row[3]

Row id:         1
City:           athens
County:         greece
Population:     1368
Row id:         2
City:           bangkok
County:         thailand
Population:     1178
Row id:         3
City:           barcelona
County:         spain
Population:     1280
Row id:         4
City:           berlin
County:         east_germany
Population:     3481
Row id:         5
City:           birmingham
County:         united_kingdom
Population:     1112
Row id:         6
City:           bombay
County:         india
Population:     2839
Row id:         7
City:           brussels
County:         belgium
Population:     986
Row id:         8
City:           bucharest
County:         romania
Population:     1237
Row id:         9
City:           budapest
County:         hungary
Population:     1757
Row id:         10
City:           buenos_aires
County:         argentina
Population:     3404
Row id:         11
City:           cairo
County:         egypt
Population:     2373
Row id:         12
City: 

# It Works !

## This script was made into module so it could be imported

In [ ]:
%%writefile SdbINFO.py
import sqlite3
def Sdbinfo(unknown, storage):
    con = sqlite3.connect(storage)
    co = con.cursor()
    co.execute("""
    CREATE VIRTUAL TABLE IF NOT EXISTS data 
    USING FTS4(unknown, row, columns);
    """)
    con.commit()
    
    conn = sqlite3.connect(unknown)
    conn.text_factory = str
    c = conn.cursor()
    res = c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    row = c.fetchone()
    row = str(row)
    row = row.replace("(","");row = row.replace(",)","")
    row = row.replace("'","");row = row.replace(",","")
    cur = c.execute("select * from \"%s\"" %  row)
    columns = [description[0] for description in cur.description]
    col = str(columns)
    col = col.replace("[","");col = col.replace("]","")
    coll = col.replace("'","")
    print row, coll
    co.execute("INSERT into data (unknown, row, columns) values (?,?,?)",(unknown, row, coll))
    con.commit()
    conn.close()
    conn.close()
    return columns

# Using the SdbINFO module 
## Read the database.list file and retrieve the information ( tables and columns )
## from the listed databases . . . 
## THEN post it all in a database for easy retieval

In [55]:
import SdbINFO
fname = "database.list"
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    total = i + 1
    return total

def readAll(fname):
    f = open(fname , 'r' )
    lines = f.readlines()
    f.close()
    return lines

def file_head(filein):
    count = 0
    fn = 0
    with open(filein) as fin:
        for fline in fin:
            fline = str(fline)
            # the first line of an SQLITE data file reads " SQLite format 3@ "
            # this splits the line at the @ 
            fin = fline.split("@")
            for fi in fin:
                # This line seeks the string SQLite, That will identify the file as an SQLite database
                if count < 2 and "SQLite" in fi:
                    if len(fi) < 25:
                        return filein 
        

tl = file_len(fname)
#limit_memory(1000)
print "Total Lines : ",tl

for line in readAll(fname):
    line = line.replace("\n", "")
    sqlite = file_head(line)
    try:
        if len(sqlite) >6:
            
            storage = "NEW.db"
            database = sqlite.replace("\n", "")
            print database
            SdbINFO.Sdbinfo(database, storage)
    except:
        pass



Total Lines :  209
/home/jack/unique-End2.db
ipynb file, content, description, hash
/home/jack/rollerball/Library/AssetVersioning.db
assetversion changeset, guid, name, parent, assettype, digest, oldversion, parentfolderid
/home/jack/rollerball/Library/ShaderCache.db
shadererrors guid, msg, msg2, file, plat, line, kind
/home/jack/nltk_data/corpora/city_database/city.db
city_table City, Country, Population
/home/jack/sketchbook/jack/Sqlite_FTS4/data/test.db
table_one field_one, field_two
/home/jack/sketchbook/jack/SQLite_update_by_id/data/test.db
table_one field_one, field_two
/home/jack/sketchbook/jack/Sqlite_Delete_by_id/data/test.db
table_one field_one, field_two
/home/jack/sketchbook/jack/SQLite_read/data/test.db
table_one field_one, field_two
/home/jack/sketchbook/jack/SQLite_Insert/data/test.db
table_one field_one, field_two
/home/jack/sketchbook/jack/Sqlite_Create_tables/data/test.db
table_one field_one, field_two
/home/jack/sketchbook/jack/SQLite_insert_multiline/data/test.db
ta

/home/jack/Desktop/TEXT_NLTK/text_stuff/postit2.db
ipynb title, line, keyword
/home/jack/Desktop/TEXT_NLTK/text_stuff/database-bak/pdfs.db
pdfs text, title
/home/jack/Desktop/TEXT_NLTK/text_stuff/database-bak/notebooks.db
ipynb title, line
/home/jack/Desktop/TEXT_NLTK/text_stuff/database-bak/notebooks-1.db
ipynb title, line
/home/jack/Desktop/TEXT_NLTK/text_stuff/database-bak/collection.db
tweets text, account
/home/jack/Desktop/Snippet_Warehouse/ipydb64.db
python code, keyword
/home/jack/Desktop/Snippet_Warehouse/ipydb.db
python code, keyword
/home/jack/Desktop/Snippet_Warehouse/Default.db
storage id, storeOne, storeTwo, keywords
/home/jack/Desktop/pattern-master/examples/08-server/02-api/rate.db
rate key, path, limit, time
/home/jack/Desktop/Flask-Login-App-Tutorial/Static/User.db
USERS USERNAME, PASSWORD
/home/jack/Desktop/CFDG/Context-Free-in-Jupyter-Notebook-BAK/data/sqlite-01.db
python code, keyword
/home/jack/Desktop/CFDG/data/sqlite-01.db
python code, keyword
/home/jack/Desktop

# Open the Database created from "database.list"
# and View the Info Gathered

In [56]:
storage = "NEW.db"
import sqlite3
con = sqlite3.connect(storage)
co = con.cursor()
for row in co.execute("SELECT ROWID, * from data"):
    print "ROWID : ",row[0]
    print "DATABASE : ",row[1]
    print "TABLE : ",row[2]
    print "COLUMNS : ",row[3]
    print "-----------------"

con.close()

ROWID :  1
DATABASE :  /home/jack/unique-End2.db
TABLE :  ipynb
COLUMNS :  file, content, description, hash
-----------------
ROWID :  2
DATABASE :  /home/jack/rollerball/Library/AssetVersioning.db
TABLE :  assetversion
COLUMNS :  changeset, guid, name, parent, assettype, digest, oldversion, parentfolderid
-----------------
ROWID :  3
DATABASE :  /home/jack/rollerball/Library/ShaderCache.db
TABLE :  shadererrors
COLUMNS :  guid, msg, msg2, file, plat, line, kind
-----------------
ROWID :  4
DATABASE :  /home/jack/nltk_data/corpora/city_database/city.db
TABLE :  city_table
COLUMNS :  City, Country, Population
-----------------
ROWID :  5
DATABASE :  /home/jack/sketchbook/jack/Sqlite_FTS4/data/test.db
TABLE :  table_one
COLUMNS :  field_one, field_two
-----------------
ROWID :  6
DATABASE :  /home/jack/sketchbook/jack/SQLite_update_by_id/data/test.db
TABLE :  table_one
COLUMNS :  field_one, field_two
-----------------
ROWID :  7
DATABASE :  /home/jack/sketchbook/jack/Sqlite_Delete_by_id/

In [21]:
# /home/jack/rollerball/Library/AssetVersioning.db
# assetversion changeset, guid, name, parent, assettype, digest, oldversion, parentfolderid
# this is a databse created by reading RSS feeds
database = "/home/jack/hubiC/Databases/bigfeedfts.db"
import sqlite3
con = sqlite3.connect(database)
co = con.cursor()
count = 0
for row in co.execute("SELECT ROWID, * from  bbctech"):
    print "ROWID : ",row[0]
    print "RSS HEAD :  ",row[1]
    print "FEED :      ",row[2]
    print "-----------------"
    count = count + 1
    if count >4:
        break

con.close()

ROWID :  1
RSS HEAD :   (u'BBC News - Technology', u'http://www.bbc.co.uk/news/', u'http://www.bbc.co.uk/news/technology-41913740')
FEED :       u"Facebook 'remembers' nude images to combat revenge porn: http://www.bbc.co.uk/news/technology-41913740"
-----------------
ROWID :  2
RSS HEAD :   (u'BBC News - Technology', u'http://www.bbc.co.uk/news/', u'http://www.bbc.co.uk/news/technology-41913740')
FEED :       u'Google promoted Texas gunman fake tweets: http://www.bbc.co.uk/news/technology-41915065'
-----------------
ROWID :  3
RSS HEAD :   (u'BBC News - Technology', u'http://www.bbc.co.uk/news/', u'http://www.bbc.co.uk/news/technology-41913740')
FEED :       u'Twitter to expand 280-character tweets: http://www.bbc.co.uk/news/technology-41900880'
-----------------
ROWID :  4
RSS HEAD :   (u'BBC News - Technology', u'http://www.bbc.co.uk/news/', u'http://www.bbc.co.uk/news/technology-41913740')
FEED :       u"Facebook's fake news experiment backfires: http://www.bbc.co.uk/news/technolog

# ----------------------------------------------

# Bonus stuff - The cells below were tools 
# used to create the completed functional cells above

# Bonus stuff - The cells below were tools 
# used to create the completed functional cells above

In [46]:
#import resource

def limit_memory(maxsize):
    soft, hard = resource.getrlimit(resource.RLIMIT_AS)
    resource.setrlimit(resource.RLIMIT_AS, (maxsize, hard))
fname = "database.list"
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    total = i + 1
    return total

def readAll(fname):
    f = open(fname , 'r' )
    lines = f.readlines()
    f.close()
    return lines

def file_head(filein):
    count = 0
    fn = 0
    with open(filein) as fin:
        for fline in fin:
            fline = str(fline)
            # the first line of an SQLITE data file reads " SQLite format 3@ "
            # this splits the line at the @ 
            fin = fline.split("@")
            for fi in fin:
                # This line seeks the string SQLite, That will identify the file as an SQLite database
                if count < 2 and "SQLite" in fi:
                    if len(fi) < 25:
                        return filein 
        

tl = file_len(fname)
#limit_memory(1000)
print "Total Lines : ",tl

for line in readAll(fname):
    line = line.replace("\n", "")
    sqlite = file_head(line)
    try:
        if len(sqlite) >6:
            print sqlite
    except:
        pass



Total Lines :  209
/home/jack/unique-End2.db
/home/jack/rollerball/Library/AssetVersioning.db
/home/jack/rollerball/Library/ShaderCache.db
/home/jack/nltk_data/corpora/city_database/city.db
/home/jack/sketchbook/jack/Sqlite_FTS4/data/test.db
/home/jack/sketchbook/jack/SQLite_update_by_id/data/test.db
/home/jack/sketchbook/jack/Sqlite_Delete_by_id/data/test.db
/home/jack/sketchbook/jack/SQLite_read/data/test.db
/home/jack/sketchbook/jack/SQLite_Insert/data/test.db
/home/jack/sketchbook/jack/Sqlite_Create_tables/data/test.db
/home/jack/sketchbook/jack/SQLite_insert_multiline/data/test.db
/home/jack/sketchbook/libraries/BezierSQLib/examples/MySQL_example3/data/test.db
/home/jack/sketchbook/libraries/BezierSQLib/examples/SQLite_example2/data/test.db
/home/jack/sketchbook/libraries/BezierSQLib/examples/SQLite_example1/data/test.db
/home/jack/notebooks/ipython-notebooks-master/notebooks/misc/misc.db
/home/jack/New Unity Project/Library/AssetVersioning.db
/home/jack/Desktop/thedb.db
/home/jac

In [56]:
import SdbINFO
database = "/home/jack/unique-End2.db"
storage = "DBinfo.db"
SdbINFO.Sdbinfo(database, storage)

ipynb file, content, description, hash


['file', 'content', 'description', 'hash']

In [61]:
def file_head(filein):
    count = 0
    with open(filein) as fin:
        for fline in fin:
            while count < 1:
                print count,fline
                count = count+1
            return    
filein = "/home/jack/unique-End2.db" 
file_head(filein)

���������������������}wqke_YSMGA;5/)#



In [62]:
def file_head(filein):
    count = 0
    with open(filein) as fin:
        for fline in fin:
            while count < 1:
                print count,fline
                count = count+1
            return    
filein = "/home/jack/rollerball/Library/AssetVersioning.db" 
file_head(filein)

0 SQLite format 3   @                                                                    -�   �    ��)� �                                                                         d!!�tablerow_countsrow_countsCREATE TABLE row_counts ( 	id		integer primary key, 	rows	integer )q''�!tableconfigurationconfigurationCREATE TABLE configuration ( 	guid		blob primary key, 	changeset	integer )9M' indexsqlite_autoindex_configuration_1configuration��UtablechangesetchangesetCREATE TABLE changeset ( 	id			integer primary key,	message		text, 	owner		text, 	date_time	integer)k''�tableparentfoldersparentfoldersCREATE TABLE parentfolders (	id			integer primary key,	folder		text)�v%%�/tableassetversionassetversionCREATE TABLE assetversion ( 	changeset	integer,	guid	blob,	name	text collate nocase,	parent	blob,	assettype integer,	digest	blob,  	oldversion	integer,  	parentfolderid integer, primary key (changeset, guid) )7K% indexsqlite_autoindex_asset

In [39]:
#import resource

def limit_memory(maxsize):
    soft, hard = resource.getrlimit(resource.RLIMIT_AS)
    resource.setrlimit(resource.RLIMIT_AS, (maxsize, hard))
fname = "database.list"
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    total = i + 1
    return total

def readAll(fname):
    f = open(fname , 'r' )
    lines = f.readlines()
    f.close()
    return lines

def file_head(filein):
    count = 0
    fn = 0
    with open(filein) as fin:
        for fline in fin:
            fline = str(fline)
            # the first line of an SQLITE data file reads " SQLite format 3@ "
            # this splits the line at the @ 
            fin = fline.split("@")
            for fi in fin:
                # This line seeks the string SQLite, That will identify the file as an SQLite database
                if count < 2 and "SQLite" in fi:
                    if len(fi) < 25:
                        print filein
                        print fi
                        print "----------------------------"  
                        count = count+1
                        return filein 
        

tl = file_len(fname)
#limit_memory(1000)
print "Total Lines : ",tl

for line in readAll(fname):
    line = line.replace("\n", "")
    file_head(line)

Total Lines :  209
/home/jack/unique-End2.db
SQLite format 3   
----------------------------
/home/jack/rollerball/Library/AssetVersioning.db
SQLite format 3   
----------------------------
/home/jack/rollerball/Library/ShaderCache.db
SQLite format 3   
----------------------------
/home/jack/nltk_data/corpora/city_database/city.db
SQLite format 3   
----------------------------
/home/jack/sketchbook/jack/Sqlite_FTS4/data/test.db
SQLite format 3   
----------------------------
/home/jack/sketchbook/jack/SQLite_update_by_id/data/test.db
SQLite format 3   
----------------------------
/home/jack/sketchbook/jack/Sqlite_Delete_by_id/data/test.db
SQLite format 3   
----------------------------
/home/jack/sketchbook/jack/SQLite_read/data/test.db
SQLite format 3   
----------------------------
/home/jack/sketchbook/jack/SQLite_Insert/data/test.db
SQLite format 3   
----------------------------
/home/jack/sketchbook/jack/Sqlite_Create_tables/data/test.db
SQLite forma

/home/jack/Desktop/Snippet_Warehouse/ipydb64.db
SQLite format 3   
----------------------------
/home/jack/Desktop/Snippet_Warehouse/ipydb.db
SQLite format 3   
----------------------------
/home/jack/Desktop/Snippet_Warehouse/Default.db
SQLite format 3   
----------------------------
/home/jack/Desktop/pattern-master/examples/08-server/02-api/rate.db
SQLite format 3   
----------------------------
/home/jack/Desktop/Flask-Login-App-Tutorial/Static/User.db
SQLite format 3   
----------------------------
/home/jack/Desktop/CFDG/Context-Free-in-Jupyter-Notebook-BAK/data/sqlite-01.db
SQLite format 3   
----------------------------
/home/jack/Desktop/CFDG/data/sqlite-01.db
SQLite format 3   
----------------------------
/home/jack/Desktop/CFDG/data/main.db
SQLite format 3   
----------------------------
/home/jack/Desktop/GRAPHICS/newdatabase.db
SQLite format 3   
----------------------------
/home/jack/Desktop/data/processing.db
SQLite format 3   
-----------

In [13]:
from time import sleep
def file_head(filein):
    count = 0
    with open(filein) as fin:
        for fline in fin:
            fline = str(fline)
            fin = fline.split(" ")
            print "\n",filein,"\n________________\n"
            for fi in fin:
                if count < 4:
                    print count,fi             
                    count = count+1
            return  
            
filein = "/home/jack/unique-End2.db"            
file_head(filein)            


/home/jack/unique-End2.db 
________________

0 SQLite
1 format
2 3   @
3 


In [ ]:

fname = "database.list"
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    total = i + 1
    return total

def readAll(fname):
    f = open(fname , 'r' )
    lines = f.readlines()
    f.close()
    return lines

def file_head(filein):
    count = 0
    with open(filein) as fin:
        for fline in fin:
            fline = str(fline)
            fin = fline.split(" ")
            for fi in fin:
                while count < 10:
                    print "\n",filein,"\n",count,fi,"\n________________\n"
                    count = count+1
                return    
        

tl = file_len(fname)
print "Total Lines : ",tl
for line in readAll(fname):
    line = line.replace("\n", "")
    file_head(line)

In [ ]:
f = open( 'database.list' , 'r' )
line = f.readline()
while line :
    print line
    line = f.readline()
f.close()

In [36]:
import SdbINFO13
from time import sleep
import sys
def main():
    title = "database.list"
    filein = open(title,"r").readlines()
    fn = 0
    while databases in filein:
        count = 0
        fn = fn+1
        database = str(databases)
        database = database.replace("\n","")
        unknown = database
        print "1 : ",unknown
        while count < 6:
            count =0
            with open((unknown).decode('utf8')) as infile:
                for lines in infile:
                    line = lines.split(" ")
                    for li in line:

                        sleep(.5)
                        print fn,"/",count," : ",li
                        count = count +1
                        
                        
main()                        
                        

1 :  /home/jack/unique-End2.db
1 / 0  :  SQLite
1 / 1  :  format
1 / 2  :  3   @
1 / 3  :  
��B                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

KeyboardInterrupt: 

In [25]:
import SdbINFO13
from time import sleep
import sys
count = 0
storage = "Exp.db"
title = "database.list"
filein = open(title,"r").readlines()
for databases in filein:
    database = str(databases)
    database = database.replace("\n","")
    unknown = database
    print unknown
    SdbINFO13.Sdbinfo(unknown, storage)
    sleep(.5)
    count = count +1
    if count > 10:
        break

/home/jack/unique-End2.db
ipynb file, content, description, hash
/home/jack/rollerball/Library/AssetVersioning.db
assetversion changeset, guid, name, parent, assettype, digest, oldversion, parentfolderid
/home/jack/rollerball/Library/ShaderCache.db
shadererrors guid, msg, msg2, file, plat, line, kind
/home/jack/nltk_data/corpora/city_database/city.db
city_table City, Country, Population
/home/jack/nltk_data/corpora/toolbox/iu_mien_samp.db
/home/jack/nltk_data/corpora/toolbox/MDF/MDFSampl.db
/home/jack/nltk_data/corpora/toolbox/MDF/MDF_AltH.db
/home/jack/sketchbook/jack/Sqlite_FTS4/data/test.db
table_one field_one, field_two
/home/jack/sketchbook/jack/SQLite_update_by_id/data/test.db
table_one field_one, field_two
/home/jack/sketchbook/jack/Sqlite_Delete_by_id/data/test.db
table_one field_one, field_two
/home/jack/sketchbook/jack/SQLite_read/data/test.db
table_one field_one, field_two


# Where I discovered not all *.db were SQLITE databases

In [27]:
import SdbINFO
from time import sleep
title = "database.list"
for database in open(title,"r").readlines():
    try:
        storage = "DBinfo.db"
    except OperationalError:
        print title," is not a database."
        pass

    
    database = database.replace("\n","")
    #database = '/home/jack/Desktop/databases/PDE2db.db'
    #databasx = "/home/jack/Desktop/databases/PDE2db.db"
    #cur = c.execute("select * from \"%s\"" %  row)
    #SdbINFO.Sdbinfo("?", storage),(database)
    unknown = database
    print unknown

    SdbINFO.Sdbinfo(unknown, storage)
    sleep(.5)


/home/jack/unique-End2.db
ipynb file, content, description, hash
/home/jack/rollerball/Library/AssetVersioning.db
assetversion changeset, guid, name, parent, assettype, digest, oldversion, parentfolderid
/home/jack/rollerball/Library/ShaderCache.db
shadererrors guid, msg, msg2, file, plat, line, kind
/home/jack/nltk_data/corpora/city_database/city.db
city_table City, Country, Population
/home/jack/nltk_data/corpora/toolbox/iu_mien_samp.db


DatabaseError: file is not a database

In [55]:
!ls /home/jack/unique-End2.db

/home/jack/unique-End2.db


In [31]:
test = "bunch o junk"
total = "this is \'%s\'" % test
print total

this is 'bunch o junk'


In [28]:
test = "bunch o junk"
total = "this is '?'",(test) 
print total[0],total[1]

this is '?' bunch o junk


In [56]:
import SdbINFO
database = "/home/jack/unique-End2.db"
storage = "DBinfo.db"
SdbINFO.Sdbinfo(database, storage)

ipynb file, content, description, hash


['file', 'content', 'description', 'hash']

# Get the size of the Databases listed in database.list

In [16]:
import os
count = 0
fn = 0
title = "database.list"
for lines in open(title,"r").readlines():
    fn = fn + 1
    filename = os.path.basename(lines)
    lines = lines.replace("\n","")
    b = os.path.getsize(lines)
    a = float(b/1000)
    filename = filename.replace("\n","")
    print fn," :",a,"k  -",lines
    count = count +1

1  : 7005179.0 k  - /home/jack/unique-End2.db
2  : 14.0 k  - /home/jack/rollerball/Library/AssetVersioning.db
3  : 3.0 k  - /home/jack/rollerball/Library/ShaderCache.db
4  : 4.0 k  - /home/jack/nltk_data/corpora/city_database/city.db
5  : 1.0 k  - /home/jack/nltk_data/corpora/toolbox/iu_mien_samp.db
6  : 29.0 k  - /home/jack/nltk_data/corpora/toolbox/MDF/MDFSampl.db
7  : 0.0 k  - /home/jack/nltk_data/corpora/toolbox/MDF/MDF_AltH.db
8  : 2.0 k  - /home/jack/sketchbook/jack/Sqlite_FTS4/data/test.db
9  : 2.0 k  - /home/jack/sketchbook/jack/SQLite_update_by_id/data/test.db
10  : 2.0 k  - /home/jack/sketchbook/jack/Sqlite_Delete_by_id/data/test.db
11  : 2.0 k  - /home/jack/sketchbook/jack/SQLite_read/data/test.db
12  : 2.0 k  - /home/jack/sketchbook/jack/SQLite_Insert/data/test.db
13  : 2.0 k  - /home/jack/sketchbook/jack/Sqlite_Create_tables/data/test.db
14  : 2.0 k  - /home/jack/sketchbook/jack/SQLite_insert_multiline/data/test.db
15  : 2.0 k  - /home/jack/sketchbook/libraries/BezierSQLib

171  : 10.0 k  - /home/jack/anaconda2/pkgs/blaze-0.11.3-py27h5f341da_0/lib/python2.7/site-packages/blaze/examples/data/iris.db
172  : 539.0 k  - /home/jack/anaconda2/lib/python2.7/site-packages/blaze/examples/data/teams.db
173  : 10.0 k  - /home/jack/anaconda2/lib/python2.7/site-packages/blaze/examples/data/iris.db
174  : 28.0 k  - /home/jack/database/mysqlite.db
175  : 12.0 k  - /home/jack/hubiC/unqlite-1/UnQ_01.db
176  : 12.0 k  - /home/jack/hubiC/unqlite-1/cool.db
177  : 12.0 k  - /home/jack/hubiC/unqlite-1/test.db
178  : 12.0 k  - /home/jack/hubiC/unqlite-1/Uncool.db
179  : 28.0 k  - /home/jack/hubiC/Databases/sniptest.db
180  : 20.0 k  - /home/jack/hubiC/Databases/test1.db
181  : 13217.0 k  - /home/jack/hubiC/Databases/pde2db.db
182  : 90.0 k  - /home/jack/hubiC/Databases/pages.db
183  : 3360919.0 k  - /home/jack/hubiC/Databases/nov19b.db
184  : 192.0 k  - /home/jack/hubiC/Databases/runit.db
185  : 20.0 k  - /home/jack/hubiC/Databases/pdefts4.db
186  : 20.0 k  - /home/jack/hubiC/D

In [3]:
from time import sleep
import sys
count = 0
title = "/home/jack/nltk_data/corpora/city_database/city.db"
# ��j!!�tablecity_tablecity_tableCREATE TABLE city_table

title = "/home/jack/Desktop/databases/PDE2db.db"
#SQLite format 3@  f�
#tablepde_docsizepde_docsizeCREATE TABLE 'pde_docsize'(docid INTEGER P

title = "/home/jack/rollerball/Library/AssetVersioning.db"
SQLite format 3@  -����)��d!!�tablerow_countsrow_countCREATE TABLE row_counts 

#title = "/home/jack/nltk_data/corpora/city_database/city.db"
#title = "/home/jack/nltk_data/corpora/city_database/city.db"
for lines in open(title,"r").readlines():
    sleep(.5)
    print lines
    count = count +1
    if count > 6:sys.exit()

SQLite format 3   @                                                                    -�   �    ��)� �                                                                         d!!�tablerow_countsrow_countCREATE TABLE row_counts ( 	id		integer primary key, 	rows	integer )q''�!tableconfigurationconfigurationCREATE TABLE configuration ( 	guid		blob primary key, 	changeset	integer )9M' indexsqlite_autoindex_configuration_1configuration��UtablechangesetchangesetCREATE TABLE changeset ( 	id			integer primary key,	message		text, 	owner		text, 	date_time	integer)k''�tableparentfoldersparentfoldersCREATE TABLE parentfolders (	id			integer primary key,	folder		text)�v%%�/tableassetversionassetversionCREATE TABLE assetversion ( 	changeset	integer,	guid	blob,	name	text collate nocase,	parent	blob,	assettype integer,	digest	blob,  	oldversion	integer,  	parentfolderid integer, primary key (changeset, guid) )7K% indexsqlite_autoindex_assetvers

SystemExit: 

In [28]:
from time import sleep
import sys
count = 0
title = "database.list"
for lines in open(title,"r").readlines():
    sleep(.5)
    print lines
    count = count +1
    if count > 6:
        break

/home/jack/unique-End2.db

/home/jack/rollerball/Library/AssetVersioning.db

/home/jack/rollerball/Library/ShaderCache.db

/home/jack/nltk_data/corpora/city_database/city.db

/home/jack/nltk_data/corpora/toolbox/iu_mien_samp.db

/home/jack/nltk_data/corpora/toolbox/MDF/MDFSampl.db

/home/jack/nltk_data/corpora/toolbox/MDF/MDF_AltH.db



## Remove the new line (  \n )
## if the \n exists in the final read the file will not open
Example :
    the file will be read as:<br />
    /home/jack/nltk_data/corpora/city_database/city.db/n
    
That will result in the file  /n  does not exist     

In [64]:
from time import sleep
import sys
count = 0
title = "database.list"
for lines in open(title,"r").readlines():
    sleep(.5)
    print lines
    lines = lines.replace("\n","")
    count = count +1
    if count > 6:sys.exit()

/home/jack/unique-End2.db

/home/jack/rollerball/Library/AssetVersioning.db

/home/jack/rollerball/Library/ShaderCache.db

/home/jack/nltk_data/corpora/city_database/city.db

/home/jack/nltk_data/corpora/toolbox/iu_mien_samp.db

/home/jack/nltk_data/corpora/toolbox/MDF/MDFSampl.db

/home/jack/nltk_data/corpora/toolbox/MDF/MDF_AltH.db



SystemExit: 